In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),write_options={"use_spark": True}
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-27 00:04:26,477 INFO: Initializing external client
2025-05-27 00:04:26,479 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:04:29,279 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057
Fetching data from 2025-04-28 04:04:26.352547+00:00 to 2025-05-27 03:04:26.352547+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.12s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-04-28 05:00:00+00:00,2,0
1,2025-04-28 06:00:00+00:00,2,0
2,2025-04-28 07:00:00+00:00,2,0
3,2025-04-28 08:00:00+00:00,2,0
4,2025-04-28 09:00:00+00:00,2,0
...,...,...,...
177220,2025-05-26 23:00:00+00:00,263,17
177221,2025-05-27 00:00:00+00:00,263,11
177222,2025-05-27 01:00:00+00:00,263,6
177223,2025-05-27 02:00:00+00:00,263,7


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177225 entries, 0 to 177734
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         177225 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  177225 non-null  int32                  
 2   rides               177225 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.1 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177225 entries, 0 to 177734
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         177225 non-null  datetime64[us]
 1   pickup_location_id  177225 non-null  int32         
 2   rides               177225 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.1 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,129,20,110,5,27,84,155,5,97,144,...,151,29,98,115,14,176,58,121,263,2025-05-14 00:00:00
1,0,2,5,3,7,3,2,6,3,2,...,0,3,8,0,1,2,8,0,25,2025-04-28 14:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,150,2025-04-28 13:00:00
3,0,1,0,2,1,0,0,1,0,0,...,1,1,0,0,1,0,1,0,56,2025-05-21 19:00:00
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,32,2025-05-16 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,1,0,0,0,0,1,1,0,1,...,0,1,0,0,0,0,0,0,180,2025-05-22 02:00:00
251,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,27,2025-05-03 07:00:00
252,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,126,2025-05-23 21:00:00
253,3,0,0,0,0,0,0,0,1,0,...,2,0,1,0,0,1,0,3,182,2025-05-01 22:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-27 00:04:55,169 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-27 00:04:55,183 INFO: Initializing external client
2025-05-27 00:04:55,184 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:04:57,235 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057
Fetching data from 2025-04-28 04:04:55.169285+00:00 to 2025-05-27 03:04:55.169285+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.54s) 


In [12]:
current_date

Timestamp('2025-05-27 04:04:55.169285+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-05-26 05:00:00
1,0,2,0,0,0,1,1,1,0,0,...,0,1,0,0,1,0,0,0,3,2025-05-26 05:00:00
2,3,5,3,9,7,4,3,1,2,3,...,3,7,10,9,1,9,2,0,4,2025-05-26 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-05-26 05:00:00
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,6,2025-05-26 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,0,1,1,0,0,0,1,0,0,...,1,0,1,0,0,0,1,0,259,2025-05-26 05:00:00
251,0,2,0,5,3,4,2,2,2,1,...,7,7,3,6,1,3,3,0,260,2025-05-26 05:00:00
252,3,0,4,13,19,15,15,48,41,52,...,14,15,8,3,3,2,3,1,261,2025-05-26 05:00:00
253,25,75,143,144,130,109,91,79,79,87,...,15,19,10,8,3,1,2,1,262,2025-05-26 05:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-27 00:05:25,766 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-27 00:05:25,791 INFO: Initializing external client
2025-05-27 00:05:25,796 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:05:28,063 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057


Downloading: 0.000%|          | 0/313830 elapsed<00:00 remaining<?

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,2.0
3,5,0.0
4,6,0.0
...,...,...
250,259,0.0
251,260,1.0
252,261,2.0
253,262,16.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132,  48, 186, 100, 162, 170, 263, 230,  68, 238])